In [42]:
locations = ["kandy","bentota","nuwara eliya"]
duration = 4

import pandas as pd
itinerary = pd.read_csv("iternary 1(in).csv", encoding='ISO-8859-1')
itinerary.head()

,tour_code,tour_title,nights,days,day,location,activity,Typical visit time
0,4N5DKNEBC,04 Nights_05 Days_KANDY Nuwaraeliya BENTOTA CO...,4,5,1,KANDY,PINNAWELA Elephant Orphanage,Morning
1,4N5DKNEBC,04 Nights_05 Days_KANDY Nuwaraeliya BENTOTA CO...,4,5,1,KANDY,Cultural dance show,Evening
2,4N5DKNEBC,04 Nights_05 Days_KANDY Nuwaraeliya BENTOTA CO...,4,5,2,NUWARA ELIYA,KANDY Temple,Morning
3,4N5DKNEBC,04 Nights_05 Days_KANDY Nuwaraeliya BENTOTA CO...,4,5,2,NUWARA ELIYA,Peradeniya Botanical Gardens,Morning
4,4N5DKNEBC,04 Nights_05 Days_KANDY Nuwaraeliya BENTOTA CO...,4,5,2,NUWARA ELIYA,Gem Museum,Morning


In [43]:
import pandas as pd
def calculate_location_percentage_by_tour(itinerary_file, locations, duration):
    itinerary = pd.read_csv(itinerary_file, encoding='ISO-8859-1')
    itinerary['location'] = itinerary['location'].str.lower()
    locations = [loc.lower() for loc in locations]
    filtered_itinerary = itinerary[itinerary['days'] == duration]
    
    tour_location_percentages = {}
    
    for tour_title, tour_data in itinerary.groupby('tour_title'):
        unique_tour_locations = tour_data['location'].unique().tolist()
        
        matching_locations = [loc for loc in locations if loc in unique_tour_locations]
        if len(unique_tour_locations) > len(locations):
          percentage = (len(matching_locations) / len(unique_tour_locations)) * 100
        else:
          percentage = (len(locations) / len(unique_tour_locations)) * 100
        
        tour_location_percentages[tour_title] = {
            'percentage': percentage,
            'matched_locations': matching_locations
        }
    
    return tour_location_percentages

def find_best_tour(results):
    best_tour = None
    min_difference = 100

    for tour, data in results.items():
      difference = abs(data['percentage'] - 100)
      if difference == 0:
        return tour
      if difference < min_difference:
        min_difference = difference
        best_tour = tour
      elif difference == min_difference and data['percentage'] > (results.get(best_tour, {})).get('percentage',0):
        best_tour = tour
    return best_tour

def calculate_day_counts_by_location(itinerary_file, locations, tour_title):
    itinerary = pd.read_csv(itinerary_file, encoding='ISO-8859-1')
    
    itinerary['location'] = itinerary['location'].str.lower()

    filtered_itinerary = itinerary[itinerary['tour_title'] == tour_title]
    
    unique_days = filtered_itinerary.drop_duplicates(['location','day'])

    location_day_count = unique_days.groupby("location")["day"].count()

    return location_day_count.to_dict()

def find_best_tour_with_days(itinerary_file, locations, duration):
    result = calculate_location_percentage_by_tour(itinerary_file, locations, duration)
    best_match = find_best_tour(result)
    day_counts = calculate_day_counts_by_location(itinerary_file, locations, best_match)
    
    print(f"Day counts & locations for the best matched tour :'{best_match}':")
    print(day_counts)
    
    # Initialize location days with 0
    location_days = {loc.lower(): 0 for loc in locations}
    
    current_duration = 0
    selected_locations = []

    sorted_locations = sorted(day_counts.items(), key=lambda x: x[1], reverse=True)
    
    # First pass: allocate full days to matching locations
    for location, days in sorted_locations:
        if location in location_days:
            # Allocate full days while not exceeding total duration
            allocatable_days = min(days, duration - current_duration)
            location_days[location] = allocatable_days
            current_duration += allocatable_days
            selected_locations.append((location, allocatable_days))
            
            # Break if we've reached the exact duration
            if current_duration == duration:
                break
    
    selected_location_names = [loc[0] for loc in selected_locations]
    missing_locations = [loc for loc in locations if loc not in selected_location_names]
    i=0
    while (current_duration != duration) or len(missing_locations) != 0:

      selected_cities = [city for city, _ in selected_locations]
      missing_locations = [city for city in locations if city not in selected_cities]
      if current_duration < duration:
        lowest_location = min(selected_locations, key=lambda x: x[1])
        index = selected_locations.index(lowest_location)
        selected_locations[index] = (lowest_location[0], lowest_location[1] + 1)
        current_duration += 1
      elif current_duration > duration:
        lowest_location = max(selected_locations, key=lambda x: x[1])
        index = selected_locations.index(lowest_location)
        selected_locations[index] = (lowest_location[0], lowest_location[1] - 1)
        current_duration -= 1

      if len(missing_locations) != 0:
        if current_duration == duration:
          lowest_location = max(selected_locations, key=lambda x: x[1])
          index = selected_locations.index(lowest_location)
          selected_locations[index] = (lowest_location[0], lowest_location[1] - 1)
          current_duration -= 1

          selected_locations.append((missing_locations[0], 1))
          location_days[missing_locations[0]] += 1
          current_duration += 1
          missing_locations.pop(0)
        else:
          selected_locations.append((missing_locations[0], 1))
          location_days[missing_locations[0]] += 1
          current_duration += 1
          missing_locations.pop(0)
    return list(filter(lambda x: x[1] > 0, selected_locations))

selected_locations = find_best_tour_with_days("iternary 1(in).csv", locations, duration)
print("Final List with days")
selected_locations

Day counts & locations for the best matched tour :'04 Nights_05 Days_KANDYPINNAWELA NUWARA ELIYACOLOMBO':
{'colombo': 2, 'kandy': 2, 'nuwara eliya': 1}
Final List with days


[('kandy', 1), ('nuwara eliya', 2), ('bentota', 1)]

### Shortest path calculation. 

In [44]:
import requests
import json
import itertools
from geopy.distance import geodesic

def geocodes(city):
    headers = {
        "User-Agent": "app1/1.0 (ashen@example.com)"
    }
    try:
        response = requests.get(f"https://nominatim.openstreetmap.org/search?q={city}&format=json", headers=headers)
        
        data = json.loads(response.content)
        if data:
            latitude = float(data[0]["lat"])
            longitude = float(data[0]["lon"])
            return latitude, longitude
        else:
            print(f"No coordinates found for {city}")
            return None, None
    except Exception as e:
        print(f"Error geocoding {city}: {e}")
        return None, None

def calculate_distances(selected_locations):
    coordinates = {}
    for location, *_ in selected_locations:
        lat, lon = geocodes(location)
        if lat is not None and lon is not None:
            coordinates[location] = (lat, lon)
        else:
            print(f"Could not find coordinates for {location}")
            return None
    
    distances = {}
    for i in range(len(selected_locations)):
        for j in range(i + 1, len(selected_locations)):
            loc1, *_ = selected_locations[i]
            loc2, *_ = selected_locations[j]
            distance = geodesic(coordinates[loc1], coordinates[loc2]).km
            distances[(loc1, loc2)] = distance
            distances[(loc2, loc1)] = distance  # Add reverse direction
    
    return distances, coordinates

def find_shortest_path(selected_locations):
    # Check if Colombo is in the locations
    colombo_exists = any(loc == "colombo" for loc, *_ in selected_locations)
    
    # If Colombo is not in the list, add it
    if not colombo_exists:
        selected_locations.insert(0, ("colombo", 1))
    
    # Calculate distances between locations
    distances_result = calculate_distances(selected_locations)
    
    if distances_result is None:
        return None, None
    
    distances, coordinates = distances_result
    
    locations = [loc for loc, *_ in selected_locations]
    
    # Ensure Colombo is the first location
    start_location = "colombo"
    locations.remove(start_location)
    
    # Try all possible permutations of remaining locations
    shortest_distance = float('inf')
    shortest_path = None
    
    for path in itertools.permutations(locations):
        full_path = (start_location,) + path  # Include Colombo at the start
        
        total_distance = 0
        for i in range(len(full_path) - 1):
            total_distance += distances.get((full_path[i], full_path[i + 1]), float('inf'))
        
        if total_distance < shortest_distance:
            shortest_distance = total_distance
            shortest_path = full_path
    
    return shortest_path, shortest_distance

path, distance = find_shortest_path(selected_locations)

if path and distance:
    print("\nShortest Path:")
    print(" -> ".join(reversed(path)))
    print(f"Total Distance: {distance:.2f} km")
else:
    print("Could not find a valid path.")


Shortest Path:
kandy -> nuwara eliya -> bentota -> colombo
Total Distance: 199.61 km


## Just extracting the activities

In [47]:
itinerary['location'] = itinerary['location'].str.lower()
itinerary['activity'] = itinerary['activity'].str.lower()  # Apply lowercase to 'activity' as well


itinerary = itinerary.drop_duplicates(subset=['location', 'activity'])
itinerary = itinerary.dropna(subset=['location', 'activity'])

path = [p.lower() for p in path]

grouped_activities = itinerary.groupby("location")["activity"].apply(list)
grouped_activities = grouped_activities[grouped_activities.index.isin(path)]
print(grouped_activities)

# Create a dictionary with at most 3 activities per location
location_dict = {location: activities[:3] for location, activities in grouped_activities.items()}

location_dict



location
colombo         [colombo city tour & shopping , departure airp...
kandy           [pinnawela elephant orphanage, cultural dance ...
nuwara eliya    [kandy temple , peradeniya botanical gardens, ...
Name: activity, dtype: object


{'colombo': ['colombo city tour & shopping ',
  'departure airport drop',
  'lotus tower'],
 'kandy': ['pinnawela elephant orphanage',
  'cultural dance show ',
  'kandy temple'],
 'nuwara eliya': ['kandy temple ',
  'peradeniya botanical gardens',
  'gem museum']}

In [49]:
path

['colombo', 'bentota', 'nuwara eliya', 'kandy']

In [52]:
!pip install reportlab markdown beautifulsoup4

   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/1.9 MB 441.3 kB/s eta 0:00:04
   ---------------- ----------------------- 0.8/1.9 MB 588.4 kB/s eta 0:00:02
   ---------------- ----------------------- 0.8/1.9 MB 588.4 kB/s eta 0:00:02
   -------------------------- ------------- 1.3/1.9 MB 762.6 kB/s eta 0:00:01
   -------------------------------- ------- 1.6/1.9 MB 822.3 kB/s eta 0:00:01
   -------------------------------- ------- 1.6/1.9 MB 822.3 kB/s eta 0:00:01
   ------------------------------------- -- 1.8/1.9 MB 818.3 kB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 808.4 kB/s eta 0:00:00


In [ ]:
# from reportlab.lib import colors
# from reportlab.lib.pagesizes import letter, landscape
# from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
# from reportlab.lib.units import inch
# from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table, TableStyle, KeepTogether
# from reportlab.pdfgen import canvas
# from bs4 import BeautifulSoup
# import markdown
# import json
# from datetime import datetime
# import os

# class PDFWithHeaderFooter(SimpleDocTemplate):
#     def __init__(self, filename, logo_path=None, **kwargs):
#         SimpleDocTemplate.__init__(self, filename, **kwargs)
#         self.page_count = 0
#         self.logo_path = logo_path
        
#     def beforePage(self):
#         self.page_count += 1
#         self.canv.saveState()
        
#         # Only draw logo if path exists and is valid
#         if self.logo_path and os.path.exists(self.logo_path):
#             self.canv.drawImage(self.logo_path, 40, letter[1] - 50, width=1*inch, height=0.5*inch)
        
#         # Header setup
#         self.canv.setFillColor(colors.HexColor('#2C3E50'))
#         self.canv.setFont("Helvetica", 14)
#         self.canv.drawString(letter[0] - 200, letter[1] - 30, 
#                            f"Generated: {datetime.now().strftime('%Y-%m-%d')}")
        
#         self.canv.setStrokeColor(colors.HexColor('#BDC3C7'))
#         self.canv.line(40, letter[1] - 60, letter[0], letter[1] - 60)
        
#         # Footer setup
#         self.canv.setFont("Helvetica", 14)
#         self.canv.drawString(letter[0]/2 - 20, 30, f"Page {self.page_count}")
#         self.canv.drawString(40, 30, "Confidential")
#         self.canv.line(0, 50, letter[0]+100, 50)
#         self.canv.restoreState()

# def wrap_header(header, max_length=1):
#     """Wrap long headers into multiple lines"""
#     if not header:
#         return ""
#     words = str(header).split()
#     lines = []
#     current_line = []
    
#     for word in words:
#         if len(' '.join(current_line + [word])) <= max_length:
#             current_line.append(word)
#         else:
#             if current_line:
#                 lines.append(' '.join(current_line))
#             current_line = [word]
    
#     if current_line:
#         lines.append(' '.join(current_line))
    
#     return '\n'.join(lines)

# def process_markdown_tables(markdown_text):
#     try:
#         html = markdown.markdown(markdown_text, extensions=['tables'])
#         soup = BeautifulSoup(html, 'html.parser')
#         tables = soup.find_all('table')
        
#         if not tables:
#             return None
            
#         all_table_data = []
#         for table in tables:
#             data = []
#             headers = [wrap_header(th.get_text().strip()) for th in table.find_all('th')]
#             if not headers:  # Skip tables without headers
#                 continue
                
#             data.append(headers)
            
#             for row in table.find_all('tr')[1:]:
#                 row_data = [td.get_text().strip() for td in row.find_all('td')]
#                 if len(row_data) == len(headers):  # Only add rows that match header count
#                     data.append(row_data)
                
#             if len(data) > 1:  # Only add tables with data rows
#                 all_table_data.append(data)
            
#         return all_table_data
#     except Exception as e:
#         print(f"Error processing markdown tables: {str(e)}")
#         return None

# def create_pdf_report(content_json, output_file, logo_path=None):
#     try:
#         # Validate input
#         if not isinstance(content_json, dict) or "Report" not in content_json:
#             raise ValueError("Invalid content_json format")
            
#         doc = PDFWithHeaderFooter(
#             output_file,
#             logo_path=logo_path,
#             pagesize=landscape(letter),
#             topMargin=1.3*inch,
#             bottomMargin=1*inch,
#             leftMargin=0.5*inch,
#             rightMargin=0.5*inch
#         )
        
#         styles = getSampleStyleSheet()
#         title_style = ParagraphStyle(
#             'CustomTitle',
#             parent=styles['Heading1'],
#             fontSize=18,
#             spaceAfter=20,
#             textColor=colors.HexColor('#2C3E50')
#         )
        
#         body_style = ParagraphStyle(
#             'CustomBody',
#             parent=styles['Normal'],
#             fontSize=14,
#             spaceAfter=12,
#             leading=14
#         )
        
#         elements = []
#         elements.append(Paragraph("Cracker Products Sales Analysis Report", title_style))
#         elements.append(Paragraph("January 2024", styles['Heading2']))
#         elements.append(Spacer(1, 20))
        
#         for section in content_json["Report"]:
#             section_elements = []
#             for topic_key, topic_value in section.items():
#                 if not isinstance(topic_value, str):
#                     continue
                    
#                 if topic_key.startswith("topic"):
#                     section_elements.append(Paragraph(topic_value, title_style))
#                 elif topic_key.startswith("description"):
#                     if "topic2" in section:  # Handle markdown tables
#                         tables_data = process_markdown_tables(topic_value)
#                         if tables_data:
#                             for table_data in tables_data:
#                                 available_width = doc.width
#                                 col_count = len(table_data[0])
#                                 col_widths = [available_width/col_count] * col_count
                                
#                                 table = Table(table_data, repeatRows=1, colWidths=col_widths)
#                                 table.setStyle(TableStyle([
#                                     ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#2C3E50')),
#                                     ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
#                                     ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
#                                     ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
#                                     ('FONTSIZE', (0, 0), (-1, 0), 11),
#                                     ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
#                                     ('BACKGROUND', (0, 1), (-1, -1), colors.Color(0.95, 0.95, 0.95)),
#                                     ('TEXTCOLOR', (0, 1), (-1, -1), colors.HexColor('#2C3E50')),
#                                     ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
#                                     ('FONTSIZE', (0, 1), (-1, -1), 10),
#                                     ('GRID', (0, 0), (-1, -1), 1, colors.Color(0.8, 0.8, 0.8)),
#                                     ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
#                                     ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
#                                     ('PADDING', (0, 0), (-1, -1), 6),
#                                 ]))
#                                 section_elements.append(table)
#                                 section_elements.append(Spacer(1, 20))
#                     else:
#                         try:
#                             soup = BeautifulSoup(topic_value, 'html.parser')
#                             for element in soup.children:
#                                 if element.name == 'p':
#                                     section_elements.append(Paragraph(str(element.decode_contents()), body_style))
#                                 elif element.name == 'ul':
#                                     for li in element.find_all('li'):
#                                         section_elements.append(Paragraph("• " + li.decode_contents(), body_style))
#                         except Exception as e:
#                             print(f"Error processing HTML content: {str(e)}")
#                             section_elements.append(Paragraph(topic_value, body_style))
                    
#                     section_elements.append(Spacer(1, 10))
            
#             elements.append(KeepTogether(section_elements))
        
#         doc.build(elements)
#         return True
#     except Exception as e:
#         print(f"Error creating PDF report: {str(e)}")
#         return False
# # Use with your JSON content including markdown tables
# # Example usage
# report_content ={
#     "Report": [
#         {
#             "topic1": "Introduction",
#             "description1": "<p>This report provides a comprehensive analysis of the sales performance of various cracker products for the month of January 2024. The analysis includes key metrics such as total gross sales, average weekly penetration, weekly retail selling price (RSP), out-of-stock (OOS) rates, fill rates, and promotional activities. The goal is to identify top-performing and underperforming products and derive actionable insights.</p>"
#         },
#         {
#             "topic2": "Data",
#             "description1": """
# The increase/decrease qty, RSP, distinct item count for the 'Ambient Liquid Milk' category between '2024-05-03' and '2024-05-17'?

# | week_starting | stg_item_category_desc_txt | total_weekly_gross_sales | total_weekly_unit_sales | average_weekly_rsp | weekly_distinct_item_count | pct_change_gross_sales | pct_change_unit_sales | pct_change_average_rsp | change_in_distinct_item_count |
# | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
# | 2024-05-03 | Ambient Liquid Milk | 5.660513E7 | 129562.0 | 644.638 | 36 | NULL | NULL | NULL | NULL |
# | 2024-05-10 | Ambient Liquid Milk | 5.7065741E7 | 132682.0 | 644.814 | 36 | 0.008 | 0.024 | 0.0 | 0 |
# | 2024-05-17 | Ambient Liquid Milk | 5.0448661E7 | 118116.0 | 660.699 | 37 | -0.116 | -0.11 | 0.025 | 1 |


# The increase or decrease overall volume for the 'Ambient Liquid Milk' category between '2024-05-03' and '2024-05-17'?

# | week_starting | unit_of_measure | total_weekly_volume | percentage_change |
# | --- | --- | --- | --- |
# | 2024-05-03 | L | 100502.44 | NULL |
# | 2024-05-10 | L | 101524.7 | 1.02 |
# | 2024-05-17 | L | 89952.56 | -11.4 |
# """
#         },

#         {
#             "topic3": "Answers to Key Questions",
#             "description3": "<p>The analysis of the sales data for January 2024 reveals the following key insights:</p><ul><li><strong>Top Performing Products:</strong> The top-performing product is the 'MUNCHEE SUPER CREAM CRACKER 490G' with the highest total gross sales of 3,688,320.0 on the week starting 2024-01-19.</li><li><strong>Underperforming Products:</strong> The 'MALIBAN CRM CRACKER POCKET PACK 85G' consistently shows the lowest sales figures, with a total gross sales of 90.0 across multiple weeks.</li><li><strong>Out-of-Stock Rates:</strong> The 'MALIBAN CRACKERS THINS 95G' has the highest monthly OOS rate of 0.873 on the week starting 2024-01-05, indicating frequent stockouts.</li><li><strong>Fill Rates:</strong> The 'MUNCHEE SUPER CREAM CRACKER 490G' achieved a perfect fill rate of 1.0 on the week starting 2024-01-19, ensuring product availability.</li></ul>"
#         },
#         {
#             "topic4": "Key Insights & Analysis",
#             "description4": "<p>Based on the data, several key insights can be drawn:</p><ul><li><strong>Sales Performance:</strong> The 'MUNCHEE SUPER CREAM CRACKER 490G' consistently performs well, indicating strong market demand and effective distribution strategies.</li><li><strong>Stock Management:</strong> High OOS rates for certain products like 'MALIBAN CRACKERS THINS 95G' suggest issues in inventory management that need to be addressed to prevent lost sales opportunities.</li><li><strong>Product Availability:</strong> Products with high fill rates, such as 'MUNCHEE SUPER CREAM CRACKER 490G', demonstrate efficient supply chain operations, ensuring customer satisfaction.</li><li><strong>Promotional Activities:</strong> The absence of promotional activities across all products suggests potential areas for growth through targeted marketing campaigns.</li></ul>"
#         },
#         {
#             "topic5": "Conclusion & Recommendations",
#             "description5": "<p>In conclusion, the analysis highlights the importance of effective stock management and the potential benefits of promotional activities. The 'MUNCHEE SUPER CREAM CRACKER 490G' serves as a benchmark for successful product performance, while underperforming products like 'MALIBAN CRM CRACKER POCKET PACK 85G' require strategic interventions to boost sales. Recommendations include:</p><ul><li>Implementing targeted promotional campaigns to increase product visibility and sales.</li><li>Improving inventory management practices to reduce OOS rates and enhance product availability.</li><li>Conducting further analysis to identify factors contributing to the success of top-performing products and replicating these strategies across other product lines.</li></ul>"
#         },
#         {
#             "topic6": "Further Areas of Investigation",
#             "description6": "<p>To validate the conclusions and recommendations, further areas of investigation include:</p><ul><li>Analyzing customer feedback and preferences to understand the factors driving product performance.</li><li>Conducting competitor analysis to identify market trends and opportunities for differentiation.</li><li>Evaluating the impact of promotional activities on sales performance through controlled experiments.</li><li>Assessing the effectiveness of supply chain operations and identifying areas for improvement to enhance product availability and reduce OOS rates.</li></ul>"
#         }
#     ]
# }

# create_pdf_report(report_content, 'sales_analysis_report.pdf',logo_path="octave.png")

True

In [6]:
from reportlab.lib import colors
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch, mm
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, KeepTogether
from reportlab.pdfgen import canvas
import markdown
from bs4 import BeautifulSoup
from datetime import datetime
import json

class PDFWithHeaderFooter(SimpleDocTemplate):
    def __init__(self, filename, **kwargs):
        SimpleDocTemplate.__init__(self, filename, **kwargs)
        self.page_count = 0
        
    def beforePage(self):
        self.page_count += 1
        self.canv.saveState()
        
        # Header with logo
        logo_path = "octave.png"
        try:
            self.canv.drawImage(logo_path, 20*mm, A4[1] - 25*mm, width=25*mm, height=12.5*mm)
        except:
            self.canv.setFont("Helvetica-Bold", 20)
            self.canv.drawString(20*mm, A4[1] - 25*mm, "COMPANY")
        
        # Header text
        self.canv.setFont("Helvetica", 9)
        self.canv.drawString(A4[0] - 80*mm, A4[1] - 15*mm,
                          f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
        
        self.canv.line(20*mm, A4[1] - 30*mm, A4[0] - 20*mm, A4[1] - 30*mm)
        
        # Footer
        self.canv.setFont("Helvetica", 9)
        footer_text = f"Page {self.page_count}"
        self.canv.drawString(A4[0]/2 - 10*mm, 15*mm, footer_text)
        self.canv.drawString(20*mm, 15*mm, "Confidential")
        
        self.canv.line(20*mm, 25*mm, A4[0] - 20*mm, 25*mm)
        self.canv.restoreState()

def process_markdown_tables(markdown_text):
    """Process markdown text and extract tables"""
    html = markdown.markdown(markdown_text, extensions=['tables'])
    soup = BeautifulSoup(html, 'html.parser')
    
    tables = []
    for table in soup.find_all('table'):
        data = []
        headers = [Paragraph(th.get_text().strip(), ParagraphStyle('Header', wordWrap='CJK')) 
                  for th in table.find_all('th')]
        if headers:
            data.append(headers)
        
        for row in table.find_all('tr'):
            if not row.find_all('th'):  # Skip header row
                row_data = [Paragraph(td.get_text().strip(), ParagraphStyle('Cell', wordWrap='CJK')) 
                           for td in row.find_all('td')]
                if row_data:
                    data.append(row_data)
        
        tables.append(data)
        table.decompose()
    
    remaining_text = soup.get_text().strip()
    return remaining_text, tables

def process_html_content(html_content):
    """Process HTML content and convert to reportlab paragraphs"""
    soup = BeautifulSoup(html_content, 'html.parser')
    elements = []
    styles = getSampleStyleSheet()
    
    for element in soup.find_all(['p', 'ul']):
        if element.name == 'p':
            elements.append(Paragraph(str(element), styles['Normal']))
        elif element.name == 'ul':
            for li in element.find_all('li'):
                bullet_text = f"• {li.get_text()}"
                elements.append(Paragraph(bullet_text, styles['Normal']))
    
    return elements

def create_table_style():
    """Create consistent table style"""
    return TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 12),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),
        ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 1), (-1, -1), 10),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('PADDING', (0, 0), (-1, -1), 6),
        ('WORDWRAP', (0, 0), (-1, -1), True),
        ('LEFTPADDING', (0, 0), (-1, -1), 3*mm),
        ('RIGHTPADDING', (0, 0), (-1, -1), 3*mm),
    ])

def generate_sales_report_pdf(json_data, output_pdf):
    # Calculate usable width (A4 width minus margins)
    margin = 20*mm  # 20mm margins on each side
    usable_width = A4[0] - 2*margin
    
    doc = PDFWithHeaderFooter(
        output_pdf,
        pagesize=A4,
        topMargin=35*mm,
        bottomMargin=30*mm,
        leftMargin=margin,
        rightMargin=margin
    )
    
    styles = getSampleStyleSheet()
    elements = []
    
    # Title
    title_style = ParagraphStyle(
        'CustomTitle',
        parent=styles['Title'],
        fontSize=24,
        spaceAfter=30
    )
    elements.append(Paragraph("Sales Performance Analysis Report", title_style))
    
    # Topic style
    topic_style = ParagraphStyle(
        'TopicStyle',
        parent=styles['Heading1'],
        fontSize=18,
        spaceAfter=20,
        spaceBefore=30
    )
    
    for i, section in enumerate(json_data, 1):
        # Get the topic and description keys
        topic_key = next(key for key in section.keys() if key.startswith('topic'))
        desc_key = next(key for key in section.keys() if key.startswith('description'))
        
        # Create section elements
        section_elements = []
        
        # Add topic heading
        topic_text = section[topic_key]
        section_elements.append(Paragraph(topic_text, topic_style))
        
        description = section[desc_key]
        
        # Check if content is markdown (contains table markers)
        if '|' in description and '---' in description:
            # Process markdown content
            text, tables = process_markdown_tables(description)
            if text:
                section_elements.append(Paragraph(text, styles['Normal']))
            
            # Add tables
            for table_data in tables:
                if table_data:
                    # Calculate column widths to fit within usable width
                    num_cols = len(table_data[0])
                    col_width = usable_width / num_cols
                    col_widths = [col_width] * num_cols
                    
                    table = Table(table_data, colWidths=col_widths, repeatRows=1)
                    table.setStyle(create_table_style())
                    # Always wrap tables in KeepTogether
                    section_elements.append(KeepTogether(table))
                    section_elements.append(Spacer(1, 20))
        else:
            # Process HTML content
            html_elements = process_html_content(description)
            section_elements.extend(html_elements)
        
        section_elements.append(Spacer(1, 20))
        
        # If it's the second section, don't wrap in KeepTogether
        if i == 2:
            elements.extend(section_elements)
        else:
            # Wrap all other sections in KeepTogether
            elements.append(KeepTogether(section_elements))
    
    doc.build(elements)
if __name__ == "__main__":
    json_data = [
        {
            "topic1": "Introduction",
            "description1": "<p>This report provides a comprehensive analysis of the sales performance of various cracker products for the month of January 2024. The analysis includes key metrics such as total gross sales, average weekly penetration, weekly retail selling price (RSP), out-of-stock (OOS) rates, fill rates, and promotional activities. The goal is to identify top-performing and underperforming products and derive actionable insights.</p>"
        },
         {
            "topic3": "Answers to Key Questions",
            "description3": "<p>The analysis of the sales data for January 2024 reveals the following key insights:</p><ul><li><strong>Top Performing Products:</strong> The top-performing product is the 'MUNCHEE SUPER CREAM CRACKER 490G' with the highest total gross sales of 3,688,320.0 on the week starting 2024-01-19.</li><li><strong>Underperforming Products:</strong> The 'MALIBAN CRM CRACKER POCKET PACK 85G' consistently shows the lowest sales figures, with a total gross sales of 90.0 across multiple weeks.</li><li><strong>Out-of-Stock Rates:</strong> The 'MALIBAN CRACKERS THINS 95G' has the highest monthly OOS rate of 0.873 on the week starting 2024-01-05, indicating frequent stockouts.</li><li><strong>Fill Rates:</strong> The 'MUNCHEE SUPER CREAM CRACKER 490G' achieved a perfect fill rate of 1.0 on the week starting 2024-01-19, ensuring product availability.</li></ul>"
        },
        {
            "topic2": "Data Representation",
            "description1": """ 
            The increase/decrease qty, RSP, distinct item count for the 'Ambient Liquid Milk' category between '2024-05-03' and '2024-05-17'?

| week_starting | stg_item_category_desc_txt | total_weekly_gross_sales | total_weekly_unit_sales | average_weekly_rsp | weekly_distinct_item_count | pct_change_gross_sales | pct_change_unit_sales | pct_change_average_rsp | change_in_distinct_item_count |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 2024-05-03 | Ambient Liquid Milk | 5.660513E7 | 129562.0 | 644.638 | 36 | NULL | NULL | NULL | NULL |
| 2024-05-10 | Ambient Liquid Milk | 5.7065741E7 | 132682.0 | 644.814 | 36 | 0.008 | 0.024 | 0.0 | 0 |
| 2024-05-17 | Ambient Liquid Milk | 5.0448661E7 | 118116.0 | 660.699 | 37 | -0.116 | -0.11 | 0.025 | 1 |

| week_starting | stg_item_category_desc_txt | total_weekly_gross_sales | total_weekly_unit_sales | average_weekly_rsp | weekly_distinct_item_count | pct_change_gross_sales | pct_change_unit_sales | pct_change_average_rsp | change_in_distinct_item_count |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 2024-05-03 | Ambient Liquid Milk | 5.660513E7 | 129562.0 | 644.638 | 36 | NULL | NULL | NULL | NULL |
| 2024-05-10 | Ambient Liquid Milk | 5.7065741E7 | 132682.0 | 644.814 | 36 | 0.008 | 0.024 | 0.0 | 0 |
| 2024-05-17 | Ambient Liquid Milk | 5.0448661E7 | 118116.0 | 660.699 | 37 | -0.116 | -0.11 | 0.025 | 1 |

| week_starting | stg_item_category_desc_txt | total_weekly_gross_sales | total_weekly_unit_sales | average_weekly_rsp | weekly_distinct_item_count | pct_change_gross_sales | pct_change_unit_sales | pct_change_average_rsp | change_in_distinct_item_count |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 2024-05-03 | Ambient Liquid Milk | 5.660513E7 | 129562.0 | 644.638 | 36 | NULL | NULL | NULL | NULL |
| 2024-05-10 | Ambient Liquid Milk | 5.7065741E7 | 132682.0 | 644.814 | 36 | 0.008 | 0.024 | 0.0 | 0 |
| 2024-05-17 | Ambient Liquid Milk | 5.0448661E7 | 118116.0 | 660.699 | 37 | -0.116 | -0.11 | 0.025 | 1 |

The increase or decrease overall volume for the 'Ambient Liquid Milk' category between '2024-05-03' and '2024-05-17'?

| week_starting | unit_of_measure | total_weekly_volume | percentage_change |
| --- | --- | --- | --- |
| 2024-05-03 | L | 100502.44 | NULL |
| 2024-05-10 | L | 101524.7 | 1.02 |
| 2024-05-17 | L | 89952.56 | -11.4 |

The increase/decrease qty, RSP, distinct item count for the 'Ambient Liquid Milk' category between '2024-05-03' and '2024-05-17'?

| week_starting | stg_item_category_desc_txt | total_weekly_gross_sales | total_weekly_unit_sales | average_weekly_rsp | weekly_distinct_item_count | pct_change_gross_sales | pct_change_unit_sales | pct_change_average_rsp | change_in_distinct_item_count |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 2024-05-03 | Ambient Liquid Milk | 5.660513E7 | 129562.0 | 644.638 | 36 | NULL | NULL | NULL | NULL |
| 2024-05-10 | Ambient Liquid Milk | 5.7065741E7 | 132682.0 | 644.814 | 36 | 0.008 | 0.024 | 0.0 | 0 |
| 2024-05-17 | Ambient Liquid Milk | 5.0448661E7 | 118116.0 | 660.699 | 37 | -0.116 | -0.11 | 0.025 | 1 |


The increase or decrease overall volume for the 'Ambient Liquid Milk' category between '2024-05-03' and '2024-05-17'?

| week_starting | unit_of_measure | total_weekly_volume | percentage_change |
| --- | --- | --- | --- |
| 2024-05-03 | L | 100502.44 | NULL |
| 2024-05-10 | L | 101524.7 | 1.02 |
| 2024-05-17 | L | 89952.56 | -11.4 |
"""
        },

        {
            "topic3": "Answers to Key Questions",
            "description3": "<p>The analysis of the sales data for January 2024 reveals the following key insights:</p><ul><li><strong>Top Performing Products:</strong> The top-performing product is the 'MUNCHEE SUPER CREAM CRACKER 490G' with the highest total gross sales of 3,688,320.0 on the week starting 2024-01-19.</li><li><strong>Underperforming Products:</strong> The 'MALIBAN CRM CRACKER POCKET PACK 85G' consistently shows the lowest sales figures, with a total gross sales of 90.0 across multiple weeks.</li><li><strong>Out-of-Stock Rates:</strong> The 'MALIBAN CRACKERS THINS 95G' has the highest monthly OOS rate of 0.873 on the week starting 2024-01-05, indicating frequent stockouts.</li><li><strong>Fill Rates:</strong> The 'MUNCHEE SUPER CREAM CRACKER 490G' achieved a perfect fill rate of 1.0 on the week starting 2024-01-19, ensuring product availability.</li></ul>"
        },
        {
            "topic4": "Key Insights & Analysis",
            "description4": "<p>Based on the data, several key insights can be drawn:</p><ul><li><strong>Sales Performance:</strong> The 'MUNCHEE SUPER CREAM CRACKER 490G' consistently performs well, indicating strong market demand and effective distribution strategies.</li><li><strong>Stock Management:</strong> High OOS rates for certain products like 'MALIBAN CRACKERS THINS 95G' suggest issues in inventory management that need to be addressed to prevent lost sales opportunities.</li><li><strong>Product Availability:</strong> Products with high fill rates, such as 'MUNCHEE SUPER CREAM CRACKER 490G', demonstrate efficient supply chain operations, ensuring customer satisfaction.</li><li><strong>Promotional Activities:</strong> The absence of promotional activities across all products suggests potential areas for growth through targeted marketing campaigns.</li></ul>"
        },
        {
            "topic5": "Conclusion & Recommendations",
            "description5": "<p>In conclusion, the analysis highlights the importance of effective stock management and the potential benefits of promotional activities. The 'MUNCHEE SUPER CREAM CRACKER 490G' serves as a benchmark for successful product performance, while underperforming products like 'MALIBAN CRM CRACKER POCKET PACK 85G' require strategic interventions to boost sales. Recommendations include:</p><ul><li>Implementing targeted promotional campaigns to increase product visibility and sales.</li><li>Improving inventory management practices to reduce OOS rates and enhance product availability.</li><li>Conducting further analysis to identify factors contributing to the success of top-performing products and replicating these strategies across other product lines.</li></ul>"
        },
        {
            "topic6": "Further Areas of Investigation",
            "description6": "<p>To validate the conclusions and recommendations, further areas of investigation include:</p><ul><li>Analyzing customer feedback and preferences to understand the factors driving product performance.</li><li>Conducting competitor analysis to identify market trends and opportunities for differentiation.</li><li>Evaluating the impact of promotional activities on sales performance through controlled experiments.</li><li>Assessing the effectiveness of supply chain operations and identifying areas for improvement to enhance product availability and reduce OOS rates.</li></ul>"
        }
    ]


    # json_data = json.load(json_data)
    generate_sales_report_pdf(json_data, "sales_report.pdf")